### Import

In [78]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

## Add Data to Database
* Create dataframes that match db tables
* Connect to db and add the data

In [79]:
years = [2021, 2022, 2023]

df = pd.DataFrame()

for i in years:
    temp = pd.read_csv(f'csv files/cleaned_{i}.csv')
    df = pd.concat([temp, df], ignore_index=True)

In [80]:
player = df[['player_id', 'player_id_ustat', 'player_name_futbin',
     'player_name_ustat', 'dob', 'nation', 'position']].drop_duplicates(subset=['player_id'])

# Have only 1 entry per team and remove messed up team names (FIFA 23, will fix later)
club = df[['club_id', 'club_name_futbin', 'club_name_ustat']].drop_duplicates(subset=['club_id'])
club = club[club['club_name_futbin'] != '\n                                           Controlled                                       ']

# competition = df

player_stats = df[['player_id', 'season', 'comp_id', 'club_id', 'apps',
                   'minutes', 'goals', 'assists', 'xG', 'xA']].drop_duplicates(subset=['player_id', 'season', 'comp_id', 'club_id'])

# team_stats = df[['team_id',

# comp_stats = df

player_ratings = df[['player_id', 'season', 'card_id', 'rating', 'pac', 'acceleration', 'sprint_speed', 'sho', 'positioning', 
                     'finishing', 'shot_power', 'long_shots', 'volleys', 'penalties', 'pas', 'vision', 'crossing', 
                     'fk_accuracy', 'short_passing', 'long_passing', 'curve', 'dri', 'agility', 'balance',
                     'reactions', 'ball_control', 'dribbling', 'composure', 'def', 'interceptions', 'heading_acc', 
                     'def_awareness', 'standing_tackle', 'sliding_tackle', 'phy', 'jumping', 'stamina', 
                     'strength', 'aggression', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 
                     'gk_speed', 'gk_positioning']].drop_duplicates(subset=['player_id', 'season'])

In [82]:
player_ratings

,player_id,season,card_id,rating,pac,acceleration,sprint_speed,sho,positioning,finishing,...,jumping,stamina,strength,aggression,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning
0,253290,23,51930,75,81.0,82.0,81.0,62.0,66.0,65.0,...,77.0,81.0,65.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN
1,256035,23,51890,75,93.0,92.0,93.0,69.0,68.0,72.0,...,77.0,70.0,60.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN
2,241928,23,51875,75,66.0,63.0,69.0,58.0,65.0,53.0,...,71.0,79.0,63.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN
3,201118,23,51870,75,71.0,76.0,66.0,61.0,69.0,54.0,...,72.0,66.0,57.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN
4,242964,23,51855,75,80.0,78.0,82.0,69.0,68.0,68.0,...,58.0,75.0,56.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3906,189596,21,578,86,68.0,68.0,68.0,82.0,94.0,85.0,...,77.0,86.0,66.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN
3907,233049,21,563,87,83.0,86.0,81.0,74.0,83.0,81.0,...,51.0,77.0,67.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN
3908,212622,21,552,88,71.0,77.0,66.0,72.0,80.0,66.0,...,79.0,95.0,67.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN
3909,167495,21,548,89,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,87.0,87.0,91.0,89.0,56.0,60.0


In [85]:
user = 'postgres'
password = 'password'
host = 'localhost'
db = 'postgres'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{db}')

dfs = [player, club, player_stats, player_ratings]
tables = ['player', 'club', 'player_stats', 'player_ratings']

for i in range(len(dfs)):
    dfs[i].to_sql(tables[i], con=engine, if_exists='append', index=False)

InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table player because other objects depend on it
DETAIL:  constraint fk1_player_stats on table player_stats depends on table player
constraint fk1_player_stats on table player_ratings depends on table player
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE player]
(Background on this error at: https://sqlalche.me/e/20/2j85)